In [1]:
from mxnet import gluon
import mxnet as mx
from mxnet import nd
from mxnet.gluon import nn 
import numpy as np
from tqdm import tqdm

/home/jiancheng/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
from tmpref.model import Deeplabv3

Using TensorFlow backend.
/home/jiancheng/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:766: DeprecationWarning: builtin type EagerTensor has no __module__ attribute
  EagerTensor = c_api.TFE_Py_InitEagerTensor(_EagerTensorBase)
/home/jiancheng/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/jiancheng/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


In [3]:
import keras.backend as K
import tensorflow as tf
from keras.models import Model

In [4]:
def get_gpu_session(ratio=None, interactive=False):
    config = tf.ConfigProto(allow_soft_placement=True)
    if ratio is None:
        config.gpu_options.allow_growth = True
    else:
        config.gpu_options.per_process_gpu_memory_fraction = ratio
    if interactive:
        sess = tf.InteractiveSession(config=config)
    else:
        sess = tf.Session(config=config)
    return sess


def set_gpu_usage(ratio=None):
    sess = get_gpu_session(ratio)
    K.set_session(sess)

In [5]:
set_gpu_usage()

In [6]:
keras_model = Deeplabv3(backbone="xception")

/home/jiancheng/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


In [7]:
keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 256, 256, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 256, 256, 32) 128         entry_flow_conv1_1[0][0]         
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 256, 32) 0           entry_flow_conv1_1_BN[0][0]      
__________________________________________________________________________________________________
entry_flow

In [8]:
# from keras.utils import plot_model
# plot_model(keras_model,to_file='xception.jpg',show_shapes=True)

In [9]:
for layer in keras_model.layers:
    print(layer.name)

input_1
entry_flow_conv1_1
entry_flow_conv1_1_BN
activation_1
entry_flow_conv1_2
entry_flow_conv1_2_BN
activation_2
activation_3
entry_flow_block1_separable_conv1_depthwise
entry_flow_block1_separable_conv1_depthwise_BN
entry_flow_block1_separable_conv1_pointwise
entry_flow_block1_separable_conv1_pointwise_BN
activation_4
entry_flow_block1_separable_conv2_depthwise
entry_flow_block1_separable_conv2_depthwise_BN
entry_flow_block1_separable_conv2_pointwise
entry_flow_block1_separable_conv2_pointwise_BN
zero_padding2d_1
activation_5
entry_flow_block1_separable_conv3_depthwise
entry_flow_block1_separable_conv3_depthwise_BN
zero_padding2d_2
entry_flow_block1_separable_conv3_pointwise
entry_flow_block1_shortcut
entry_flow_block1_separable_conv3_pointwise_BN
entry_flow_block1_shortcut_BN
add_1
activation_6
entry_flow_block2_separable_conv1_depthwise
entry_flow_block2_separable_conv1_depthwise_BN
entry_flow_block2_separable_conv1_pointwise
entry_flow_block2_separable_conv1_pointwise_BN
activat

In [10]:
nn.Conv2D(kernel_size=(5,4),channels=3,in_channels=3,groups=3).weight

Parameter conv0_weight (shape=(3, 1, 5, 4), dtype=<class 'numpy.float32'>)

In [11]:
from keras.layers import Input, Conv2D,DepthwiseConv2D

In [12]:
inputs = Input((10,10,3))
# conv1 = Conv2D(20,kernel_size=(5,4))(inputs)
conv1 = DepthwiseConv2D(kernel_size=(5,4))(inputs)

In [13]:
Model(inputs, conv1).weights

[<tf.Variable 'depthwise_conv2d_1/depthwise_kernel:0' shape=(5, 4, 3, 1) dtype=float32_ref>,
 <tf.Variable 'depthwise_conv2d_1/bias:0' shape=(3,) dtype=float32_ref>]

In [14]:
def compute_same_padding(kernel_size, dilation):
    #TODO: compute `same` padding for stride<=2 ? 
    kernel_size_effective = kernel_size + (kernel_size - 1) * (dilation - 1)
    pad_total = kernel_size_effective - 1
    pad_beg = pad_total // 2
    pad_end = pad_total - pad_beg
    return pad_beg, pad_end

In [15]:
class SeparableConv(nn.HybridBlock):
    #TODO: should I fix epsilon in BN layer?
    
    def __init__(self, out_filters, kernel_size, strides, dilation, depth_activation, in_filters=None, prefix=None):
        super(SeparableConv,self).__init__(prefix=prefix)
        
        if in_filters is None:
            in_filters = out_filters
        
        self.depth_activation = depth_activation

        padding = compute_same_padding(kernel_size, dilation)
        # filter_in==filter_out
        with self.name_scope():
            self.depthwise_conv = nn.Conv2D(in_filters, in_channels=in_filters, groups=in_filters, 
                                            dilation=dilation,use_bias=False,
                                            padding=padding, strides=strides,
                                            kernel_size=kernel_size,prefix='depthwise_')
            self.bn1 = nn.BatchNorm(axis=1,prefix='depthwise_BN_')
            self.pointwise_conv = nn.Conv2D(out_filters, kernel_size=1,use_bias=False, prefix='pointwise_')
            self.bn2 = nn.BatchNorm(axis=1,prefix='pointwise_BN_')
        
    def hybrid_forward(self, F, x):
        if not self.depth_activation:
            x = F.relu(x)
        x = self.depthwise_conv(x)
        x = self.bn1(x)
        if self.depth_activation:
            x = F.relu(x)
        x = self.pointwise_conv(x)
        x = self.bn2(x)
        if self.depth_activation:
            x = F.relu(x)
        return x   


In [16]:
# ?nn.BatchNorm(axis=1)

In [17]:
inputs = nd.random_normal(shape=(8,16,48,48),ctx=mx.gpu(0))

In [18]:
sep_conv = SeparableConv(16,3,1,1,False)

In [19]:
sep_conv.initialize(ctx=mx.gpu(0))

In [20]:
sep_conv(inputs).shape

(8, 16, 48, 48)

In [21]:
# ?nn.Conv2D

In [22]:
sep_conv.depthwise_conv.name

'separableconv0_depthwise'

In [23]:
class XceptionBlock(nn.HybridBlock):
    
    def __init__(self, filters_list, kernel_size, strides, dilation, depth_activation, in_filters=None, 
                 use_shortcut_conv=None, prefix=None, return_skip=False, use_shortcut=True):
        super(XceptionBlock,self).__init__(prefix=prefix)
        
        assert len(filters_list)==3
        
        if in_filters is None:
            in_filters = filters_list[0]
            
        if use_shortcut_conv is None:
            use_shortcut_conv=strides>1
        
        with self.name_scope():
            self.conv1 = SeparableConv(filters_list[0], kernel_size, 1, dilation, depth_activation,
                                       in_filters=in_filters, prefix='separable_conv1_')
            self.conv2 = SeparableConv(filters_list[1], kernel_size, 1, dilation, depth_activation,
                                       in_filters=filters_list[0],prefix='separable_conv2_')
            self.conv3 = SeparableConv(filters_list[2], kernel_size, strides, dilation, depth_activation,
                                       in_filters=filters_list[1],prefix='separable_conv3_')
            if use_shortcut_conv:
                self.shortcut_conv = nn.Conv2D(filters_list[2], kernel_size=1, strides=strides,use_bias=False,prefix='shortcut_')
                self.shortcut_bn = nn.BatchNorm(axis=1,prefix='shortcut_BN_')
        
        self.use_shortcut_conv = use_shortcut_conv
        self.use_shortcut = use_shortcut
        self.return_skip = return_skip
        
    def hybrid_forward(self, F, x):
        shortcut = x
        x = self.conv1(x)
        x = self.conv2(x)
        skip = x
        x = self.conv3(x)
        if self.use_shortcut_conv:
            shortcut = self.shortcut_conv(shortcut)
            shortcut = self.shortcut_bn(shortcut)
        if self.return_skip:
            return x+shortcut, skip
        if self.use_shortcut:
            return x+shortcut
        return x

In [24]:
class EntryFlow(nn.HybridBlock):
    
    def __init__(self, prefix='entry_flow_'):
        super(EntryFlow,self).__init__(prefix)

        with self.name_scope():
            self.conv1 = nn.HybridSequential(prefix='conv1_')
            with self.conv1.name_scope():
                self.conv1.add(nn.Conv2D(32, kernel_size=3, strides=2, padding=1,use_bias=False,prefix='1_'))
                self.conv1.add(nn.BatchNorm(axis=1,prefix='1_BN_'))
                self.conv1.add(nn.Activation("relu"))
            self.conv2 = nn.HybridSequential(prefix='conv1_')
            with self.conv2.name_scope():
                self.conv2.add(nn.Conv2D(64, kernel_size=3, padding=1,use_bias=False,prefix='2_'))
                self.conv2.add(nn.BatchNorm(axis=1,prefix='2_BN_'))
                self.conv2.add(nn.Activation("relu"))

            self.conv3 = XceptionBlock(filters_list=[128,128,128], kernel_size=3, strides=2,
                                       dilation=1, depth_activation=False,in_filters=64,prefix='block1_')
            self.conv4 = XceptionBlock(filters_list=[256,256,256], kernel_size=3, strides=2,return_skip=True,
                                       dilation=1, depth_activation=False, in_filters=128,prefix='block2_')
            self.conv5 = XceptionBlock(filters_list=[728,728,728], kernel_size=3, strides=2,
                                       dilation=1, depth_activation=False, in_filters=256,prefix='block3_')
        
    def hybrid_forward(self, F, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x, skip = self.conv4(x)
        x = self.conv5(x)
        return x, skip

In [25]:
entry_flow = EntryFlow()
entry_flow.initialize(ctx=mx.gpu())

In [26]:
middle_flow = nn.HybridSequential(prefix="middle_flow_")
with middle_flow.name_scope():
    for i in range(16):
        middle_flow.add(XceptionBlock(filters_list=[728,728,728], kernel_size=3, strides=1,
                                      dilation=1, depth_activation=False,in_filters=728,
                                      prefix="unit_%s_"%(i+1)))
middle_flow.initialize(ctx=mx.gpu(0))

In [27]:
exit_flow = nn.HybridSequential(prefix="exit_flow_")
with exit_flow.name_scope():
    exit_flow.add(XceptionBlock(filters_list=[728, 1024, 1024], kernel_size=3, strides=1,use_shortcut_conv=True,
                                      dilation=1, depth_activation=False,in_filters=728,
                                      prefix="block1_"))
    exit_flow.add(XceptionBlock(filters_list=[1536, 1536, 2048], kernel_size=3, strides=1,
                                      dilation=2, depth_activation=True, in_filters=1024,use_shortcut=False,
                                      prefix="block2_"))
exit_flow.initialize(ctx=mx.gpu(0))

In [28]:
inputs = nd.random_normal(shape=(8,3,512,512),ctx=mx.gpu(0))
entry_features, skip = entry_flow(inputs)
entry_features.shape

(8, 728, 32, 32)

In [29]:
skip.shape

(8, 256, 128, 128)

In [30]:
middle_features = middle_flow(entry_features)

In [31]:
middle_features.shape

(8, 728, 32, 32)

In [32]:
exit_features = exit_flow(middle_features)

In [33]:
exit_features.shape

(8, 2048, 32, 32)

In [34]:
exit_flow.collect_params()

exit_flow_ (
  Parameter exit_flow_block1_separable_conv1_depthwise_weight (shape=(728, 1, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter exit_flow_block1_separable_conv1_depthwise_BN_gamma (shape=(728,), dtype=<class 'numpy.float32'>)
  Parameter exit_flow_block1_separable_conv1_depthwise_BN_beta (shape=(728,), dtype=<class 'numpy.float32'>)
  Parameter exit_flow_block1_separable_conv1_depthwise_BN_running_mean (shape=(728,), dtype=<class 'numpy.float32'>)
  Parameter exit_flow_block1_separable_conv1_depthwise_BN_running_var (shape=(728,), dtype=<class 'numpy.float32'>)
  Parameter exit_flow_block1_separable_conv1_pointwise_weight (shape=(728, 728, 1, 1), dtype=<class 'numpy.float32'>)
  Parameter exit_flow_block1_separable_conv1_pointwise_BN_gamma (shape=(728,), dtype=<class 'numpy.float32'>)
  Parameter exit_flow_block1_separable_conv1_pointwise_BN_beta (shape=(728,), dtype=<class 'numpy.float32'>)
  Parameter exit_flow_block1_separable_conv1_pointwise_BN_running_mean (shape=(728

In [35]:
class PoolRecover(nn.HybridBlock):
    
    def __init__(self):
        super(PoolRecover,self).__init__()
        
        self.gap = nn.HybridSequential()
        self.gap.add(nn.GlobalAvgPool2D())
        self.gap.add(nn.Conv2D(256, kernel_size=1, use_bias=False, prefix='image_pooling_'))
        self.gap.add(nn.BatchNorm(prefix='image_pooling_BN_'))
        self.gap.add(nn.Activation("relu"))
            
    def hybrid_forward(self, F, x):
        *_, h, w = x.shape
        pool = self.gap(x)
        return F.contrib.BilinearResize2D(pool, height=h, width=w)

In [36]:
class ASPP(nn.HybridBlock):
    
    def __init__(self):
        super(ASPP, self).__init__()
        
        b0 = nn.HybridSequential()
        b0.add(nn.Conv2D(256, kernel_size=1, use_bias=False, prefix='aspp0_'))
        b0.add(nn.BatchNorm(prefix='aspp0_BN_'))
        b0.add(nn.Activation("relu"))
        
        # rate = 6 (12)
        b1 = SeparableConv(256, kernel_size=3, strides=1, dilation=6, depth_activation=True,
                           in_filters=2048, prefix='aspp1_')
        # rate = 12 (24)
        b2 = SeparableConv(256, kernel_size=3, strides=1, dilation=12, depth_activation=True,
                           in_filters=2048, prefix='aspp2_')
        # rate = 18 (36)
        b3 = SeparableConv(256, kernel_size=3, strides=1, dilation=18, depth_activation=True,
                           in_filters=2048, prefix='aspp3_')
        
        b4 = PoolRecover()
        
        self.concurent = gluon.contrib.nn.HybridConcurrent(axis=1)
        self.concurent.add(b4)
        self.concurent.add(b0)
        self.concurent.add(b1)
        self.concurent.add(b2)
        self.concurent.add(b3)
        
        self.project = nn.HybridSequential()
        self.project.add(nn.Conv2D(256, kernel_size=1, use_bias=False, prefix='concat_projection_'))
        self.project.add(nn.BatchNorm(prefix='concat_projection_BN_'))
        self.project.add(nn.Activation("relu"))
        self.project.add(nn.Dropout(0.1))

            
    def hybrid_forward(self, F, x):
        return self.project(self.concurent(x))

In [37]:
aspp = ASPP()
aspp.initialize(ctx=mx.gpu())

In [38]:
aspp(exit_features).shape

(8, 256, 32, 32)

In [39]:
aspp.collect_params()

aspp0_ (
  Parameter image_pooling_weight (shape=(256, 2048, 1, 1), dtype=<class 'numpy.float32'>)
  Parameter image_pooling_BN_gamma (shape=(256,), dtype=<class 'numpy.float32'>)
  Parameter image_pooling_BN_beta (shape=(256,), dtype=<class 'numpy.float32'>)
  Parameter image_pooling_BN_running_mean (shape=(256,), dtype=<class 'numpy.float32'>)
  Parameter image_pooling_BN_running_var (shape=(256,), dtype=<class 'numpy.float32'>)
  Parameter aspp0_weight (shape=(256, 2048, 1, 1), dtype=<class 'numpy.float32'>)
  Parameter aspp0_BN_gamma (shape=(256,), dtype=<class 'numpy.float32'>)
  Parameter aspp0_BN_beta (shape=(256,), dtype=<class 'numpy.float32'>)
  Parameter aspp0_BN_running_mean (shape=(256,), dtype=<class 'numpy.float32'>)
  Parameter aspp0_BN_running_var (shape=(256,), dtype=<class 'numpy.float32'>)
  Parameter aspp1_depthwise_weight (shape=(2048, 1, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter aspp1_depthwise_BN_gamma (shape=(2048,), dtype=<class 'numpy.float32'>)
  Par

In [40]:
keras_to_end = Model(inputs=keras_model.inputs, outputs=keras_model.get_layer("dropout_1").output)

In [41]:
cnt=0
cnt+=sum([np.prod(v.shape) for k,v in entry_flow.collect_params().items()])
cnt+=sum([np.prod(v.shape) for k,v in middle_flow.collect_params().items()])
cnt+=sum([np.prod(v.shape) for k,v in exit_flow.collect_params().items()])
cnt+=sum([np.prod(v.shape) for k,v in aspp.collect_params().items()])
cnt

41087648

In [42]:
sum([np.prod(v.shape) for k,v in middle_flow.collect_params().items()])

26033280

In [43]:
sum([np.prod(v.shape) for k,v in exit_flow.collect_params().items()])

10268560

In [44]:
keras_to_end.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 256, 256, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 256, 256, 32) 128         entry_flow_conv1_1[0][0]         
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 256, 32) 0           entry_flow_conv1_1_BN[0][0]      
__________________________________________________________________________________________________
entry_flow

In [45]:
class DeepLabv3p(nn.HybridBlock):
    
    def __init__(self, classes=21):
        super(DeepLabv3p, self).__init__()
        
        self.entry_flow = EntryFlow(prefix="entry_flow_")
        
        middle_flow = nn.HybridSequential(prefix="middle_flow_")
        with middle_flow.name_scope():
            for i in range(16):
                middle_flow.add(XceptionBlock(filters_list=[728,728,728], kernel_size=3, strides=1,
                                              dilation=1, depth_activation=False,in_filters=728,
                                              prefix="unit_%s_"%(i+1)))
        self.middle_flow = middle_flow
        
        exit_flow = nn.HybridSequential(prefix="exit_flow_")
        with exit_flow.name_scope():
            exit_flow.add(XceptionBlock(filters_list=[728, 1024, 1024], kernel_size=3, strides=1,use_shortcut_conv=True,
                                              dilation=1, depth_activation=False,in_filters=728,
                                              prefix="block1_"))
            exit_flow.add(XceptionBlock(filters_list=[1536, 1536, 2048], kernel_size=3, strides=1,
                                              dilation=2, depth_activation=True, in_filters=1024,use_shortcut=False,
                                              prefix="block2_"))
        self.exit_flow = exit_flow
        
        self.aspp = ASPP()
        
        skip_project = nn.HybridSequential()
        skip_project.add(nn.Conv2D(48, kernel_size=1, use_bias=False, prefix='feature_projection0_'))
        skip_project.add(nn.BatchNorm(prefix='feature_projection0_BN_'))
        skip_project.add(nn.Activation("relu"))
        self.skip_project = skip_project
        
        decoder = nn.HybridSequential()
        decoder.add(SeparableConv(256, kernel_size=3, strides=1, dilation=1, depth_activation=True,
                           in_filters=304, prefix='decoder_conv0_'))
        decoder.add(SeparableConv(256, kernel_size=3, strides=1, dilation=1, depth_activation=True,
                           in_filters=256, prefix='decoder_conv1_'))
        decoder.add(nn.Conv2D(classes, kernel_size=1, use_bias=True, prefix='logits_semantic_'))
        self.decoder = decoder
            
    def hybrid_forward(self, F, x):
        *_, h, w = x.shape
        
        x, skip = self.entry_flow(x)
        x = self.middle_flow(x)
        x = self.exit_flow(x)
        x = self.aspp(x)
        
        x = F.contrib.BilinearResize2D(x, height=h//4, width=w//4)
        skip = self.skip_project(skip)
        x = F.concat(x, skip, dim=1)
        
        x = self.decoder(x)
        
        return F.contrib.BilinearResize2D(x, height=h, width=w)

In [46]:
deeplab = DeepLabv3p()
deeplab.initialize(ctx=mx.gpu())

In [47]:
deeplab(inputs).shape

(8, 21, 512, 512)

In [48]:
deeplab.collect_params()

deeplabv3p0_ (
  Parameter entry_flow_conv1_1_weight (shape=(32, 3, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_1_BN_gamma (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_1_BN_beta (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_1_BN_running_mean (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_1_BN_running_var (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_2_weight (shape=(64, 32, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_2_BN_gamma (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_2_BN_beta (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_2_BN_running_mean (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_2_BN_running_var (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_depthwise_weight (shape=(64, 1, 3, 3), dtype=<class 

In [51]:
# deeplab.save_params("tmpref/deeplabv3p.params")

In [52]:
keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 256, 256, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 256, 256, 32) 128         entry_flow_conv1_1[0][0]         
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 256, 32) 0           entry_flow_conv1_1_BN[0][0]      
__________________________________________________________________________________________________
entry_flow

In [53]:
sum([np.prod(v.shape) for k,v in deeplab.collect_params().items()])

41258213

In [54]:
512/16

32.0

In [55]:
nn.Con

AttributeError: module 'mxnet.gluon.nn' has no attribute 'Con'

In [56]:
entry_flow.conv3.collect_params()

entry_flow_block1_ (
  Parameter entry_flow_block1_separable_conv1_depthwise_weight (shape=(64, 1, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_depthwise_BN_gamma (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_depthwise_BN_beta (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_depthwise_BN_running_mean (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_depthwise_BN_running_var (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_pointwise_weight (shape=(128, 64, 1, 1), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_pointwise_BN_gamma (shape=(128,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_pointwise_BN_beta (shape=(128,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_pointwise_BN_running_mean 

In [57]:
# len(weights)

In [58]:
entry_flow.collect_params()

entry_flow_ (
  Parameter entry_flow_conv1_1_weight (shape=(32, 3, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_1_BN_gamma (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_1_BN_beta (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_1_BN_running_mean (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_1_BN_running_var (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_2_weight (shape=(64, 32, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_2_BN_gamma (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_2_BN_beta (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_2_BN_running_mean (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_2_BN_running_var (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_depthwise_weight (shape=(64, 1, 3, 3), dtype=<class '

In [59]:
middle_flow.collect_params()

middle_flow_ (
  Parameter middle_flow_unit_1_separable_conv1_depthwise_weight (shape=(728, 1, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter middle_flow_unit_1_separable_conv1_depthwise_BN_gamma (shape=(728,), dtype=<class 'numpy.float32'>)
  Parameter middle_flow_unit_1_separable_conv1_depthwise_BN_beta (shape=(728,), dtype=<class 'numpy.float32'>)
  Parameter middle_flow_unit_1_separable_conv1_depthwise_BN_running_mean (shape=(728,), dtype=<class 'numpy.float32'>)
  Parameter middle_flow_unit_1_separable_conv1_depthwise_BN_running_var (shape=(728,), dtype=<class 'numpy.float32'>)
  Parameter middle_flow_unit_1_separable_conv1_pointwise_weight (shape=(728, 728, 1, 1), dtype=<class 'numpy.float32'>)
  Parameter middle_flow_unit_1_separable_conv1_pointwise_BN_gamma (shape=(728,), dtype=<class 'numpy.float32'>)
  Parameter middle_flow_unit_1_separable_conv1_pointwise_BN_beta (shape=(728,), dtype=<class 'numpy.float32'>)
  Parameter middle_flow_unit_1_separable_conv1_pointwise_BN_runn

In [60]:
keras_model.weights

[<tf.Variable 'entry_flow_conv1_1/kernel:0' shape=(3, 3, 3, 32) dtype=float32_ref>,
 <tf.Variable 'entry_flow_conv1_1_BN/gamma:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_conv1_1_BN/beta:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_conv1_2/kernel:0' shape=(3, 3, 32, 64) dtype=float32_ref>,
 <tf.Variable 'entry_flow_conv1_2_BN/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_conv1_2_BN/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_block1_separable_conv1_depthwise/depthwise_kernel:0' shape=(3, 3, 64, 1) dtype=float32_ref>,
 <tf.Variable 'entry_flow_block1_separable_conv1_depthwise_BN/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_block1_separable_conv1_depthwise_BN/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_block1_separable_conv1_pointwise/kernel:0' shape=(1, 1, 64, 128) dtype=float32_ref>,
 <tf.Variable 'entry_flow_block1_separable_conv1_pointwise_BN/gamma:0' shape=(128,) dt

In [61]:
class WeightConverter:
    
    def __init__(self, keras_model=None, weight_dict=None):
        if keras_model is None:
            self.weight_dict = weight_dict
        elif weight_dict is None:
            self.weight_dict = dict()
            for weight_var in tqdm(keras_model.weights):
                self.weight_dict[weight_var.name] = K.eval(weight_var)
        else:
            raise ValueError
            
    def replace_depthwise_weight(self, query, param):
        target = query.replace("_weight",'/depthwise_kernel:0')
        assert target in self.weight_dict, "%s->%s"%(query, target)
        weight = self.weight_dict[target]
        param.set_data(weight.transpose(2,3,0,1)) #TODO: is it right?
        
    def replace_weight(self, query, param):
        target = query.replace("_weight",'/kernel:0')
        assert target in self.weight_dict, "%s->%s"%(query, target)
        weight = self.weight_dict[target]
        param.set_data(weight.transpose(3,2,0,1))
        
    def replace_bias(self, query, param):
        target = query.replace("_bias",'/bias:0') 
        assert target in self.weight_dict, "%s->%s"%(query, target)
        weight = self.weight_dict[target]
        param.set_data(weight)
        
    def replace_gamma(self, query, param):
        target = query.replace("_gamma",'/gamma:0') 
        assert target in self.weight_dict, "%s->%s"%(query, target)
        weight = self.weight_dict[target]
        param.set_data(weight)
        
    def replace_beta(self, query, param):
        target = query.replace("_beta",'/beta:0')
        assert target in self.weight_dict, "%s->%s"%(query, target)
        weight = self.weight_dict[target]
        param.set_data(weight)
        
    def replace_running_mean(self, query, param):
        target = query.replace("_running_mean",'/moving_mean:0')
        assert target in self.weight_dict, "%s->%s"%(query, target)
        weight = self.weight_dict[target]
        param.set_data(weight)
        
    def replace_running_var(self, query, param):
        target = query.replace("_running_var",'/moving_variance:0')
        assert target in self.weight_dict, "%s->%s"%(query, target)
        weight = self.weight_dict[target]
        param.set_data(weight)
        
    def set_parameters(self, gluon_model):
        for k, param in gluon_model.collect_params().items():
            if k.endswith("depthwise_weight"):
                self.replace_depthwise_weight(k, param)
            elif k.endswith("weight"):
                self.replace_weight(k, param)
            elif k.endswith("bias"):
                self.replace_bias(k, param)
            elif k.endswith("gamma"):
                self.replace_gamma(k, param)
            elif k.endswith("beta"):
                self.replace_beta(k, param)
            elif k.endswith("running_mean"):
                self.replace_running_mean(k, param)
            elif k.endswith("running_var"):
                self.replace_running_var(k, param)
            else:
                raise ValueError

In [62]:
# converter = WeightConverter(weight_dict=weights_dict)
converter = WeightConverter(keras_model=keras_model)

100%|██████████| 732/732 [00:47<00:00, 15.52it/s]


In [64]:
converter.set_parameters(deeplab)

In [69]:
deeplab.save_params("tmpref/deeplabv3p.params")

In [70]:
type(entry_flow.collect_params())

mxnet.gluon.parameter.ParameterDict

In [71]:
converter.set_parameters(entry_flow)

In [72]:
converter.set_parameters(middle_flow)

In [73]:
converter.set_parameters(exit_flow)

In [74]:
converter.set_parameters(aspp)

In [75]:
# entry_flow.save_params("tmpref/entry_flow.params")
# middle_flow.save_params("tmpref/middle_flow.params")
# exit_flow.save_params("tmpref/exit_flow.params")
# aspp.save_params("tmpref/aspp.params")

In [282]:
xception = nn.HybridSequential()
xception.add(entry_flow)
xception.add(middle_flow)
xception.add(exit_flow)

In [283]:
xception

HybridSequential(
  (0): EntryFlow(
    (conv1): HybridSequential(
      (0): Conv2D(3 -> 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=32)
      (2): Activation(relu)
    )
    (conv2): HybridSequential(
      (0): Conv2D(32 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
      (2): Activation(relu)
    )
    (conv3): XceptionBlock(
      (conv1): SeparableConv(
        (depthwise_conv): Conv2D(1 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (bn1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
        (pointwise_conv): Conv2D(64 -> 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm(axis=1, eps=1e-05, momentu